In [106]:
import os
import gensim
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from collections import Counter
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions

In [107]:
def load_dataset(directory, label):
    data = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            review = file.read()
            data.append((review, label))
    return data

directory_path = "/Users/mrbinit/Downloads/aclImdb" 

train_pos_dir = os.path.join(directory_path, 'train', 'pos')
train_neg_dir = os.path.join(directory_path, 'train', 'neg')
test_pos_dir = os.path.join(directory_path, 'test', 'pos')
test_neg_dir = os.path.join(directory_path, 'test', 'neg')
val_pos_dir = os.path.join(directory_path, 'val', 'pos')
val_neg_dir = os.path.join(directory_path, 'val', 'neg')

train_data = load_dataset(train_pos_dir, 1) + load_dataset(train_neg_dir, 0) #1 represents positive and 0 represents negative sentiments
test_data = load_dataset(test_pos_dir, 1) + load_dataset(test_neg_dir, 0)

#split the test set into a validation set (15,000 samples) and a test set (10,000 samples)
val_data = test_data[:15000]
test_data = test_data[15000:25000]

#separate the reviews and labels from the train, test, and validation data
train_reviews, train_labels = zip(*train_data)
test_reviews, test_labels = zip(*test_data)
val_reviews, val_labels = zip(*val_data)

#check the lengths of train, test, and validation datasets
train_length = len(train_reviews)
test_length = len(test_reviews)
val_length = len(val_reviews)

print("Train dataset length:", train_length)
print("Test dataset length:", test_length)
print("Validation dataset length:", val_length)


Train dataset length: 25000
Test dataset length: 10000
Validation dataset length: 15000


In [108]:
print("Train Data:", train_reviews[:5])  
print("Test Data:", test_reviews[:5])   
print("Validation Data:", val_reviews[:5])

Train Data: ('For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.', 'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D\'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily p

In [109]:
#Regular expressions (regex) are sequences of characters that define a search pattern. They are used for string manipulation, searching, and pattern matching within text. 
import re
def has_html_tags(text):
    pattern = re.compile(r'<[^>]+>')  # Regular expression to match HTML tags
    return bool(pattern.search(text))

# Check for HTML tags in each dataset
def check_html_tags(dataset):
    for review, _ in dataset:
        if has_html_tags(review):
            return True
    return False

#check for HTML tags in each dataset
train_has_html = check_html_tags(train_data)
test_has_html = check_html_tags(test_data)
val_has_html = check_html_tags(val_data)
#print output
print("Train dataset contains HTML tags:", train_has_html)
print("Test dataset contains HTML tags:", test_has_html)
print("Validation dataset contains HTML tags:", val_has_html)


Train dataset contains HTML tags: True
Test dataset contains HTML tags: True
Validation dataset contains HTML tags: True


In [110]:
def has_url(text):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return bool(pattern.search(text))

#condition to check for url
def check_for_urls(dataset):
    for review, _ in dataset:
        if has_url(review):
            return True
    return False
#check whether there is URL or not
train_has_url = check_for_urls(train_data)
test_has_url = check_for_urls(test_data)
val_has_url = check_for_urls(val_data)

print("Train dataset contains URLs:", train_has_url)
print("Test dataset contains URLs:", test_has_url)
print("Validation dataset contains URLs:", val_has_url)

Train dataset contains URLs: True
Test dataset contains URLs: True
Validation dataset contains URLs: True


In [111]:
def has_special_characters(text):
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    return bool(pattern.search(text))

def check_for_special_characters(dataset):
    for review, _ in dataset:
        if has_special_characters(review):
            return True
    return False

train_has_special_chars = check_for_special_characters(train_data)
test_has_special_chars = check_for_special_characters(test_data)
val_has_special_chars = check_for_special_characters(val_data)
print("Train dataset contains special characters:", train_has_special_chars)
print("Test dataset contains special characters:", test_has_special_chars)
print("Validation dataset contains special characters:", val_has_special_chars)


Train dataset contains special characters: True
Test dataset contains special characters: True
Validation dataset contains special characters: True


In [112]:
#preprocess function
def preprocess_text(text):
    #remove  HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    #remove urls
    text = re.sub(r'http\S+', '', text)

    #expand contractions 
    text = contractions.fix(text)

    #tokenize 
    tokens = word_tokenize(text)

    #remove non-alphabetic tokens and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]

    #remove stopwords 
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if not word in stop_words]

    #lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

In [113]:
# Preprocess all the data
train_reviews_processed = [preprocess_text(review) for review in train_reviews]
test_reviews_processed = [preprocess_text(review) for review in test_reviews]
val_reviews_processed = [preprocess_text(review) for review in val_reviews]

/var/folders/_7/q65t_5bs3653ct73dh3s10w00000gn/T/ipykernel_4608/3997674309.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [114]:
print("Cleaned Train Data:", train_reviews_processed[:5])  
print("Cleaned Test Data:", test_reviews_processed[:5])   
print("Cleaned Validation Data:", val_reviews_processed[:5])

Cleaned Train Data: [['movie', 'get', 'respect', 'sure', 'lot', 'memorable', 'quote', 'listed', 'gem', 'imagine', 'movie', 'joe', 'piscopo', 'actually', 'funny', 'maureen', 'stapleton', 'scene', 'stealer', 'moroni', 'character', 'absolute', 'scream', 'watch', 'alan', 'skipper', 'hale', 'police', 'sgt'], ['bizarre', 'horror', 'movie', 'filled', 'famous', 'face', 'stolen', 'cristina', 'raines', 'later', 'tv', 'flamingo', 'road', 'pretty', 'somewhat', 'unstable', 'model', 'gummy', 'smile', 'slated', 'pay', 'attempted', 'suicide', 'guarding', 'gateway', 'hell', 'scene', 'raines', 'modeling', 'well', 'captured', 'mood', 'music', 'perfect', 'deborah', 'raffin', 'charming', 'cristina', 'pal', 'raines', 'move', 'creepy', 'brooklyn', 'height', 'brownstone', 'inhabited', 'blind', 'priest', 'top', 'floor', 'thing', 'really', 'start', 'cooking', 'neighbor', 'including', 'fantastically', 'wicked', 'burgess', 'meredith', 'kinky', 'couple', 'sylvia', 'mile', 'beverly', 'diabolical', 'lot', 'eli', 'wa

In [115]:
def find_max_sequence_length(train_reviews_processed):
    max_length = 0
    for tokens in train_reviews_processed:
        sequence_length = len(tokens)
        if sequence_length > max_length:
            max_length = sequence_length
    return max_length

#assuming tokenized_texts is a list of tokenized texts after preprocessing
max_sequence_length = find_max_sequence_length(train_reviews_processed)
print("Maximum sequence length:", max_sequence_length)

Maximum sequence length: 1394


In [125]:
# Train Word2Vec model
word2vec_model = gensim.models.Word2Vec(sentences=train_reviews_processed + test_reviews_processed + val_reviews_processed,
                                        vector_size=100, window=5, min_count=5, workers=4)

# Function to get vector representation of a sentence
def get_sentence_vector(tokens):
    vector = np.zeros((100,))
    count = 0
    for word in tokens:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count != 0:
        vector /= count
    return vector
    
#get sentence vector for all the dataset 
train_vectors = [get_sentence_vector(tokens) for tokens in train_reviews_processed]
test_vectors = [get_sentence_vector(tokens) for tokens in test_reviews_processed]
val_vectors = [get_sentence_vector(tokens) for tokens in val_reviews_processed]

In [126]:
#convert lists to numpy arrays 
X_train = np.array(train_vectors)
X_test = np.array(test_vectors)
X_val = np.array(val_vectors)


y_train = np.array(train_labels)
y_test = np.array(test_labels)
y_val = np.array(val_labels)

In [127]:
import tensorflow as tf
from tensorflow.keras import layers
batch_size = 32

# Define the model
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(100,)),  # Input layer with 100 features
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=20, 
                    validation_data=(X_val, y_val))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 585us/step - accuracy: 0.7789 - loss: 0.4725 - val_accuracy: 0.8283 - val_loss: 0.3822
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.8286 - loss: 0.3878 - val_accuracy: 0.8211 - val_loss: 0.4006
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.8337 - loss: 0.3809 - val_accuracy: 0.8419 - val_loss: 0.3624
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 876us/step - accuracy: 0.8354 - loss: 0.3766 - val_accuracy: 0.8717 - val_loss: 0.3104
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - accuracy: 0.8383 - loss: 0.3631 - val_accuracy: 0.8259 - val_loss: 0.3858
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - accuracy: 0.8431 - loss: 0.3607 - val_accuracy: 0.8279 - val_loss: 0.3901
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 972us/step - accuracy: 0.8457 - loss: 0.3543 - val_accuracy: 0.8279 - val_loss: 0.3895
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step - accuracy: 0.8503 - loss: 0.3444 - 

In [131]:
import keras_tuner as kt 

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value = 0, max_value = 8, default = 2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value = 16, max_value = 256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value = 16, max_value = 256)
    learning_rate = hp.Float("learning_rate", min_value = 1e-4, max_value = 1e-2 , sampling = 'log') 
    hp_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    dropout_rate = 0.2
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units = n_neurons_1 if i == 0 else n_neurons_2, activation = hp_activation))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    #output layers 
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

    # Compile the model 
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model 


In [ ]:
#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='keras_tuner_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
# print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 06s]
val_accuracy: 0.8501333594322205

Best val_accuracy So Far: 0.8871333599090576
Total elapsed time: 00h 05m 33s
Best number of hidden layers: 3
Best number of neurons per layer: 224
Best learning rate: 0.003938711556807786
Best activation function: tanh


KeyError: 'optimizer does not exist.'

In [138]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

/Users/mrbinit/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5791 - loss: 0.7012 - val_accuracy: 0.8817 - val_loss: 0.5170
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 799us/step - accuracy: 0.7639 - loss: 0.5544 - val_accuracy: 0.8917 - val_loss: 0.4171
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 993us/step - accuracy: 0.7813 - loss: 0.5183 - val_accuracy: 0.8911 - val_loss: 0.3803
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 899us/step - accuracy: 0.7953 - loss: 0.4966 - val_accuracy: 0.8909 - val_loss: 0.3710
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - accuracy: 0.8012 - loss: 0.4902 - val_accuracy: 0.8771 - val_loss: 0.3681
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - accuracy: 0.8075 - loss: 0.4860 - val_accuracy: 0.8463 - val_loss: 0.3764
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 991us/step - accuracy: 0.8096 - loss: 0.4735 - val_accuracy: 0.8721 - val_loss: 0.3503
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step - accuracy: 0.8081 - loss: 0.4759 - va

In [135]:
import keras_tuner as kt 
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    l1_reg = hp.Float("l1_reg", min_value=1e-5, max_value=1e-1, sampling='log')
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation,
                                        kernel_regularizer=regularizers.l1(l1_reg)))
        model.add(tf.keras.layers.Dropout(dropout_rate))  # Correctly accessing dropout_rate

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model

In [137]:
#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='change_tuner_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
# print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 07s]
val_accuracy: 0.8433333039283752

Best val_accuracy So Far: 0.8938000202178955
Total elapsed time: 00h 04m 59s
Best number of hidden layers: 7
Best number of neurons per layer: 125
Best learning rate: 0.0010646139883402994
Best activation function: relu
Best dropout rate: 0.4883705217384113


In [140]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val))
                        #  batch_size = 32,
                        #  callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5740 - loss: 0.7036 - val_accuracy: 0.7522 - val_loss: 0.6359
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 989us/step - accuracy: 0.7648 - loss: 0.5609 - val_accuracy: 0.7384 - val_loss: 0.6376
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.7893 - loss: 0.5213 - val_accuracy: 0.7295 - val_loss: 0.6835
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7977 - loss: 0.5053 - val_accuracy: 0.8214 - val_loss: 0.5680
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 762us/step - accuracy: 0.7982 - loss: 0.5073 - val_accuracy: 0.8212 - val_loss: 0.5697
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8044 - loss: 0.4895 - val_accuracy: 0.7567 - val_loss: 0.6488
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 766us/step - accuracy: 0.8051 - loss: 0.4961 - val_accuracy: 0.7922 - val_loss: 0.6092
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8062 - loss: 0.4931 - val_accu

In [144]:
import keras_tuner as kt 
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)

    l2_reg = hp.Float("l2_reg", min_value=1e-5, max_value=1e-1, sampling='log')  # Define L2 regularization hyperparameter
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation,
                                        kernel_regularizer=regularizers.l2(l2=l2_reg)))  # Apply L2 regularization
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [147]:
#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='l2_tuner_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
# print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 16s]
val_accuracy: 0.883733332157135

Best val_accuracy So Far: 0.8917333483695984
Total elapsed time: 00h 05m 54s
Best number of hidden layers: 6
Best number of neurons per layer: 123
Best learning rate: 0.006610896434199781
Best activation function: tanh
Best dropout rate: 0.37136854967707944


In [148]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7202 - loss: 0.8183 - val_accuracy: 0.6961 - val_loss: 0.8386
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7817 - loss: 0.6424 - val_accuracy: 0.8496 - val_loss: 0.5648
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7704 - loss: 0.7216 - val_accuracy: 0.7985 - val_loss: 0.6189
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7950 - loss: 0.6626 - val_accuracy: 0.8565 - val_loss: 0.6167
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7870 - loss: 0.6824 - val_accuracy: 0.8543 - val_loss: 0.5805
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6859 - loss: 0.7694 - val_accuracy: 0.6755 - val_loss: 0.9796
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7845 - loss: 0.7304 - val_accuracy: 0.7773 - val_loss: 0.6718


In [149]:
import keras_tuner as kt 
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    l2_reg = hp.Float("l2_reg", min_value=1e-5, max_value=1e-1, sampling='log')  # Define L2 regularization hyperparameter
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation,
                                        kernel_regularizer=regularizers.l2(l2=l2_reg)))  # Apply L2 regularization
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Define optimizer based on choice
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [150]:
#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='optimizer_tuner_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 31s]
val_accuracy: 0.8178666830062866

Best val_accuracy So Far: 0.8925333619117737
Total elapsed time: 00h 04m 52s
Best number of hidden layers: 6
Best number of neurons per layer: 17
Best learning rate: 0.0029189145650628753
Best activation function: tanh
Best optimizer: rmsprop
Best dropout rate: 0.3157570712630264


In [151]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7393 - loss: 0.6990 - val_accuracy: 0.8724 - val_loss: 0.3760
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8161 - loss: 0.4551 - val_accuracy: 0.8179 - val_loss: 0.4344
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8182 - loss: 0.4476 - val_accuracy: 0.8862 - val_loss: 0.3540
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8199 - loss: 0.4468 - val_accuracy: 0.6472 - val_loss: 0.7815
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8216 - loss: 0.4382 - val_accuracy: 0.8519 - val_loss: 0.3734
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8249 - loss: 0.4376 - val_accuracy: 0.8819 - val_loss: 0.3286
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8223 - loss: 0.4367 - val_accuracy: 0.7667 - val_loss: 0.5590
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8240 - loss: 0.4346 - val_accuracy: 0.

In [152]:
import keras_tuner as kt 
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Define optimizer based on choice
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [153]:

#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='optimizer_withoutl1_l1_tuner_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 11s]
val_accuracy: 0.8639333248138428

Best val_accuracy So Far: 0.8903999924659729
Total elapsed time: 00h 05m 45s
Best number of hidden layers: 3
Best number of neurons per layer: 187
Best learning rate: 0.00736437579965411
Best activation function: relu
Best optimizer: rmsprop
Best dropout rate: 0.3846346474970834


In [154]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7428 - loss: 0.5608 - val_accuracy: 0.7793 - val_loss: 0.5500
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8054 - loss: 0.4559 - val_accuracy: 0.7966 - val_loss: 0.5443
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8138 - loss: 0.4458 - val_accuracy: 0.7861 - val_loss: 0.5147
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8110 - loss: 0.4572 - val_accuracy: 0.8619 - val_loss: 0.3488
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8205 - loss: 0.4475 - val_accuracy: 0.7851 - val_loss: 0.4467
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8144 - loss: 0.4544 - val_accuracy: 0.8356 - val_loss: 0.4441
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8210 - loss: 0.4501 - val_accuracy: 0.8338 - val_loss: 0.4051
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8233 - loss: 0.4444 - val_accuracy: 0.

In [155]:
import keras_tuner as kt 
from tensorflow.keras import regularizers, initializers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)

    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation,
                                        kernel_initializer=initializers.HeNormal()))  # He initialization
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model with RMSprop optimizer
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [158]:

#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='initializer_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
# print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Reloading Tuner from initializer_dir/imdb_tuning/tuner0.json
Best number of hidden layers: 2
Best number of neurons per layer: 167
Best learning rate: 0.0017283418064864052
Best activation function: tanh
Best dropout rate: 0.11951765260893088


In [159]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - accuracy: 0.7769 - loss: 0.4655 - val_accuracy: 0.8155 - val_loss: 0.4070
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step - accuracy: 0.8215 - loss: 0.4016 - val_accuracy: 0.7879 - val_loss: 0.4584
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.8238 - loss: 0.3942 - val_accuracy: 0.8825 - val_loss: 0.2899
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - accuracy: 0.8226 - loss: 0.3948 - val_accuracy: 0.8811 - val_loss: 0.2934
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 878us/step - accuracy: 0.8307 - loss: 0.3868 - val_accuracy: 0.8743 - val_loss: 0.3056
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.8330 - loss: 0.3816 - val_accuracy: 0.8295 - val_loss: 0.3755
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - accuracy: 0.8297 - loss: 0.3852 - val_accuracy: 0.8428 - val_loss: 0.3683
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.8307 - loss: 0.3848 - 

In [161]:
import keras_tuner as kt 
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    l1_reg = hp.Float("l1_reg", min_value=1e-5, max_value=1e-1, sampling='log')
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation,
                                        kernel_regularizer=regularizers.l1(l1_reg)))
        model.add(tf.keras.layers.BatchNormalization())  # Add BatchNormalization layer
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [163]:

#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='batch_normalization_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
# print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 09s]
val_accuracy: 0.8147333264350891

Best val_accuracy So Far: 0.890333354473114
Total elapsed time: 00h 11m 27s
Best number of hidden layers: 5
Best number of neurons per layer: 231
Best learning rate: 0.002335347940282288
Best activation function: relu
Best dropout rate: 0.025227283264368128


In [164]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7674 - loss: 7.4119 - val_accuracy: 0.8033 - val_loss: 0.9967
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7996 - loss: 0.9009 - val_accuracy: 0.3732 - val_loss: 1.9868
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8069 - loss: 0.8350 - val_accuracy: 0.8347 - val_loss: 0.9721
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8113 - loss: 0.8070 - val_accuracy: 0.8839 - val_loss: 0.8942
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8186 - loss: 0.7472 - val_accuracy: 0.7812 - val_loss: 0.8593
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8202 - loss: 0.7982 - val_accuracy: 0.8552 - val_loss: 0.5404
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8204 - loss: 0.7534 - val_accuracy: 0.2795 - val_loss: 1.9274
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8053 - loss: 0.6653 - val_accuracy: 0.

In [165]:
import keras_tuner as kt 
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    l2_reg = hp.Float("l2_reg", min_value=1e-5, max_value=1e-1, sampling='log')  # Define L2 regularization hyperparameter
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation,
                                        kernel_regularizer=regularizers.l2(l2=l2_reg)))  # Apply L2 regularization
        model.add(tf.keras.layers.BatchNormalization())  # Add BatchNormalization layer
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Define optimizer based on choice
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [166]:

#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='batch_normalization_l2_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 12s]
val_accuracy: 0.8455333113670349

Best val_accuracy So Far: 0.8920666575431824
Total elapsed time: 00h 08m 58s
Best number of hidden layers: 3
Best number of neurons per layer: 97
Best learning rate: 0.0010741913029985602
Best activation function: relu
Best optimizer: rmsprop
Best dropout rate: 0.18241671744565546


In [167]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7563 - loss: 0.7020 - val_accuracy: 0.8421 - val_loss: 0.4934
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8113 - loss: 0.5269 - val_accuracy: 0.7817 - val_loss: 0.5076
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8174 - loss: 0.4639 - val_accuracy: 0.7495 - val_loss: 0.5795
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8287 - loss: 0.4313 - val_accuracy: 0.8561 - val_loss: 0.3936
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8273 - loss: 0.4259 - val_accuracy: 0.8632 - val_loss: 0.3872
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8284 - loss: 0.4185 - val_accuracy: 0.8034 - val_loss: 0.4692
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8279 - loss: 0.4170 - val_accuracy: 0.8315 - val_loss: 0.4281
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8307 - loss: 0.4102 - val_accuracy: 0.

In [170]:
import keras_tuner as kt
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)

    momentum = hp.Float("momentum", min_value=0.0, max_value=1.0, default=0.9)  # Define momentum hyperparameter

    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation))  # Apply L2 regularization
        # model.add(tf.keras.layers.BatchNormalization())  # Add BatchNormalization layer
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Define SGD optimizer with momentum
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [173]:

#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='momentum_no_parameter_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
# print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Reloading Tuner from momentum_no_parameter_dir/imdb_tuning/tuner0.json
Best number of hidden layers: 5
Best number of neurons per layer: 204
Best learning rate: 0.003844834195650593
Best activation function: tanh
Best dropout rate: 0.23667113519888733


In [174]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6783 - loss: 0.5802 - val_accuracy: 0.7664 - val_loss: 0.4945
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8020 - loss: 0.4404 - val_accuracy: 0.7831 - val_loss: 0.4709
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8128 - loss: 0.4120 - val_accuracy: 0.7990 - val_loss: 0.4460
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8144 - loss: 0.4123 - val_accuracy: 0.8231 - val_loss: 0.3967
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8223 - loss: 0.4035 - val_accuracy: 0.8328 - val_loss: 0.3780
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8245 - loss: 0.3984 - val_accuracy: 0.8449 - val_loss: 0.3567
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8221 - loss: 0.4019 - val_accuracy: 0.8275 - val_loss: 0.3889
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8280 - loss: 0.3963 - val_accuracy: 0.

In [172]:
import keras_tuner as kt
from tensorflow.keras import regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(100,)))
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons_1 = hp.Int("n_neurons_1", min_value=16, max_value=256)
    n_neurons_2 = hp.Int("n_neurons_2", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    optimizer_choice = hp.Choice('optimizer', values=['adagrad', 'adamax', 'nadam', 'adamW'])

    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(units=n_neurons_1 if i == 0 else n_neurons_2,
                                        activation=hp_activation))  # Apply L2 regularization
        model.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Define optimizer based on choice
    if optimizer_choice == 'adagrad':
        optimizer = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer_choice == 'adamax':
        optimizer = tf.keras.optimizers.Adamax(learning_rate=learning_rate)
    elif optimizer_choice == 'nadam':
        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    else:  # adamW
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])

    return model


In [175]:

#instanting the tuner 

tuner = kt.Hyperband(build_model, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory='other_dir',
                     project_name='imdb_tuning'
                     )

#early stopping 

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#perform the hyperparameter search 
tuner.search(X_train, y_train,
             epochs = 5,
             validation_data  = (X_val, y_val), callbacks = [stop_early])

#best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]
print("Best number of hidden layers:", best_hps.get("n_hidden"))
print("Best number of neurons per layer:", best_hps.get("n_neurons_1"))
print("Best learning rate:", best_hps.get("learning_rate"))
print("Best activation function:", best_hps.get("activation"))
print("Best optimizer:", best_hps.get("optimizer"))
print("Best dropout rate:", best_hps.get("dropout_rate"))

Trial 30 Complete [00h 00m 15s]
val_accuracy: 0.8469333052635193

Best val_accuracy So Far: 0.8851333260536194
Total elapsed time: 00h 07m 53s
Best number of hidden layers: 5
Best number of neurons per layer: 126
Best learning rate: 0.0019911924591572727
Best activation function: tanh
Best optimizer: nadam
Best dropout rate: 0.017884083469705048


In [176]:
best_model = build_model(best_hps)

# Train the model using the full training dataset
history = best_model.fit(X_train, y_train,
                         epochs=20, 
                         validation_data=(X_val, y_val),
                        #  batch_size = 32,
                         callbacks=[stop_early])  

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7908 - loss: 0.4456 - val_accuracy: 0.8152 - val_loss: 0.4058
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8279 - loss: 0.3910 - val_accuracy: 0.8443 - val_loss: 0.3692
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8296 - loss: 0.3914 - val_accuracy: 0.7693 - val_loss: 0.4916
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8276 - loss: 0.3878 - val_accuracy: 0.8675 - val_loss: 0.3269
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8295 - loss: 0.3841 - val_accuracy: 0.8416 - val_loss: 0.3604
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8307 - loss: 0.3837 - val_accuracy: 0.8596 - val_loss: 0.3320
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8321 - loss: 0.3798 - val_accuracy: 0.8541 - val_loss: 0.3619
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8348 - loss: 0.3774 - val_accuracy: 0.